# 定义 intent 和 slot

In [2]:
intent = ['看不懂的','闲聊','商品/品类','搜优惠','搜活动/会场','其他']
slot = ['日期','商品名','品牌','店铺','颜色','价格','数量','属性']
slot_dict = {'日期':'DATE','商品/品类':'PRODUCT','品牌':'BRAND','店铺':'SHOP','颜色':'COLOR','价格':'PRICE','数量':'AMOUT','属性':'ATTRIBUTE'}

In [3]:
#dict_1 = {'B-DATE': 0, 'B-PRODUCT': 1, 'B-BRAND': 2, 'B-SHOP': 3, 'B-COLOR': 4, 'B-PRICE': 5, 'B-AMOUT': 6, 'B-ATTRIBUTE': 7, 'I-DATE': 8, 'I-PRODUCT': 9, 'I-BRAND': 10, 'I-SHOP': 11, 'I-COLOR': 12, 'I-PRICE': 13, 'I-AMOUT': 14, 'I-ATTRIBUTE': 15, 'E-DATE': 16, 'E-PRODUCT': 17, 'E-BRAND': 18, 'E-SHOP': 19, 'E-COLOR': 20, 'E-PRICE': 21, 'E-AMOUT': 22, 'E-ATTRIBUTE': 23, 'O-DATE': 24, 'O-PRODUCT': 25, 'O-BRAND': 26, 'O-SHOP': 27, 'O-COLOR': 28, 'O-PRICE': 29, 'O-AMOUT': 30, 'O-ATTRIBUTE': 31, 'S-DATE': 32, 'S-PRODUCT': 33, 'S-BRAND': 34, 'S-SHOP': 35, 'S-COLOR': 36, 'S-PRICE': 37, 'S-AMOUT': 38, 'S-ATTRIBUTE': 39, '<PAD>': 40, 'O': 41}  
dict_1 = {'B-DATE': 0, 'B-PRODUCT': 1, 'B-BRAND': 2, 'B-SHOP': 3, 'B-COLOR': 4, 'B-PRICE': 5, 'B-AMOUT': 6, 'B-ATTRIBUTE': 7, 'I-DATE': 8, 'I-PRODUCT': 9, 'I-BRAND': 10, 'I-SHOP': 11, 'I-COLOR': 12, 'I-PRICE': 13, 'I-AMOUT': 14, 'I-ATTRIBUTE': 15, 'E-DATE': 16, 'E-PRODUCT': 17, 'E-BRAND': 18, 'E-SHOP': 19, 'E-COLOR': 20, 'E-PRICE': 21, 'E-AMOUT': 22, 'E-ATTRIBUTE': 23, 'S-DATE': 24, 'S-PRODUCT': 25, 'S-BRAND': 26, 'S-SHOP': 27, 'S-COLOR': 28, 'S-PRICE': 29, 'S-AMOUT': 30, 'S-ATTRIBUTE': 31, '<PAD>': 32, 'O': 33}
{0: 'B-DATE', 1: 'B-PRODUCT', 2: 'B-BRAND', 3: 'B-SHOP', 4: 'B-COLOR', 5: 'B-PRICE', 6: 'B-AMOUT', 7: 'B-ATTRIBUTE', 8: 'I-DATE', 9: 'I-PRODUCT', 10: 'I-BRAND', 11: 'I-SHOP', 12: 'I-COLOR', 13: 'I-PRICE', 14: 'I-AMOUT', 15: 'I-ATTRIBUTE', 16: 'E-DATE', 17: 'E-PRODUCT', 18: 'E-BRAND', 19: 'E-SHOP', 20: 'E-COLOR', 21: 'E-PRICE', 22: 'E-AMOUT', 23: 'E-ATTRIBUTE', 24: 'S-DATE', 25: 'S-PRODUCT', 26: 'S-BRAND', 27: 'S-SHOP', 28: 'S-COLOR', 29: 'S-PRICE', 30: 'S-AMOUT', 31: 'S-ATTRIBUTE', 32: '<PAD>', 33: 'O'}
print(dict_1)
print(type(dict_1))
for key, value in dict_1.items():
    print(key+'\t'+str(value))

In [25]:
import pandas 
import os
def read_data_by_folder(folder):
    text_list = []
    slot_list = []
    
    file_list = os.listdir(folder)
    for file_item in file_list:
        current_file_item = os.path.join(folder, file_item)
        data = pandas.read_csv(current_file_item)
        for index in data.index:
            tmp = data.loc[index].values
            text_list.append(tmp[0])

            tmp_slot_list = []
            for idx, value in enumerate(tmp):
                if idx>2 and idx %2 == 1 and idx < len(tmp)-2:
                    slot_type = tmp[idx]
                    slot_value = tmp[idx+1]
                    if len(str(slot_type).strip()) > 1:
                        try:
                            tmp_slot_list.append((slot_dict[slot_type], slot_value))
                        except:
                            tmp_slot_list.append(('ATTRIBUTE', slot_value))
            slot_list.append(tmp_slot_list)
    return text_list, slot_list

def read_clf_data_by_folder(folder):
    clf_list = []
    file_list = os.listdir(folder)
    for file_item in file_list:
        current_file_item = os.path.join(folder, file_item)
        data = pandas.read_csv(current_file_item)
        for index in data.index:
            tmp = data.loc[index].values
            clf_list.append(tmp[1]+ '\t' +tmp[0])
       
    return clf_list

# 处理slot

In [23]:
# 读 数据
text_list, slot_list = read_data_by_folder("/Users/sunhongchao/Desktop/Projects/1111/dataset/predict/")
# 写 数据
output = open('label.txt', encoding='utf-8', mode='w')

for text, tmp_slot_list in zip(text_list, slot_list):
    text_label = ['O']*len(text)
    origin_text_label = text_label.copy()
    for slot in tmp_slot_list:
        idx = text.index(slot[1])
        if idx < 0:
            continue
        slot_type = slot[0]
        
        if len(slot[1]) == 1:
            text_label[idx] = "S" + "-" + slot_type
        else:
            text_label[idx] = "B" + "-" + slot_type
        
            for idx in range(idx+1, idx+len(slot[1])-1, 1):
                text_label[idx] = "I" + "-" + slot_type
            
            text_label[idx+1] = "E" + "-" + slot_type
    
    import operator
    if operator.eq(text_label, origin_text_label) is True:
        continue

    for char, label in zip(text, text_label):
        output.write(char + '\t' + label + '\n')
    output.write('\n')
    
output.close()

# 处理意图

In [21]:
import pandas 
from sklearn.model_selection import train_test_split
import random

clf_list = read_clf_data_by_folder("/Users/sunhongchao/Desktop/Projects/1111/dataset/all_data/")
output_list = []

for item in clf_list:
    cut_list = item.split("\t")
    if len(cut_list) != 2:
        continue
    if cut_list[0] in ["其它属性", "看不懂的"]:
        continue 
    output_list.append(item)

clf_list = output_list

random.shuffle(clf_list)

cut_idx = int(len(clf_list) * 0.9)

train_clf_list = clf_list[:cut_idx]
test_clf_list = clf_list[cut_idx:]

clf_output = open('train_clf.txt', encoding='utf-8', mode='w')
for item in train_clf_list:
    clf_output.write(item + '\n')

clf_output = open('test_clf.txt', encoding='utf-8', mode='w')
for item in test_clf_list:
    clf_output.write(item + '\n')

# 数据扩充与优化方法间的关系

# Data augement for Unbalance NLU

In [26]:
import pandas 
from sklearn.model_selection import train_test_split
import random

clf_list = \
read_clf_data_by_folder("/Users/sunhongchao/Desktop/Projects/1111/dataset/all_data/")

clf_output = open('clf_promotion_activity.txt', encoding='utf-8', mode='w')
for item in clf_list:
    cut_list = item.split("\t")
    if len(cut_list) != 2:
        continue
    if cut_list[0] not in ["搜优惠", "搜活动/会场"]:
        continue 
    clf_output.write( item + '\n')


In [23]:
import synonyms
similarity_result = synonyms.nearby("明天")
print(similarity_result[0])

['明天', '谢谢', '明日', '昨天', '总有一天', '忘了', '今晚', '再来', '明明', '那时候']


In [18]:
from snownlp import SnowNLP 
for item in similarity_result[0]:                                                   
    snow_result = SnowNLP(item)                                     
    print(' '.join('%s/%s' % (word, tag) for (word, tag) in snow_result.tags))

明天/t
谢谢/v
明日/t
昨天/t
总/d 有/v 一/m 天/q
忘/v 了/y
今晚/t
再/d 来/v
明明/nr
那/r 时候/n


# 数据生成

In [ ]:
 with open('clf_promotion_activity.txt', mode='r', encoding='utf-8') as f:
     lines = f.readlines()

